In [1]:
import numpy as np
import pandas as pd
import json

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import KFold, GroupKFold
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans
import time
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
# This will tell us the columns we are predicting
pred_cols = ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']
data_dir = '../input/stanford-covid-vaccine'

In [3]:
class RMSELoss(nn.Module):
    def __init__(self, weighted, eps=1e-6):
        super().__init__()
        self.weighted = weighted
        if self.weighted:
            self.mse = nn.MSELoss(reduction='none')
        else:
            self.mse = nn.MSELoss()
        self.eps = eps

    def forward(self, y_pred, y_true, weights=None):
        loss = torch.sqrt(self.mse(y_pred, y_true) + self.eps)
        
        if self.weighted:
            loss = loss * torch.repeat_interleave(weights, repeats=68)
            loss = torch.mean(loss)
        
        return loss


class MCRMSELoss(nn.Module):
    def __init__(self, weighted=False, scored_indices=[0, 1, 2, 3, 4]):
        super().__init__()
        self.rmse = RMSELoss(weighted)
        self.idx = scored_indices

    def forward(self, y_pred, y_true, weights=None):
        score = 0.0
        for i in range(len(pred_cols)):
            if i in self.idx:
                if weights is None:
                    score += self.rmse(y_pred[:, i], y_true[:, i]) / len(self.idx)
                else:
                    score += self.rmse(y_pred[:, i], y_true[:, i], weights) / len(self.idx)

        return score

In [4]:
batch_size = 128
val_batch_size = batch_size * 8
lr = .001
nepochs = 100
nfolds = 5
nstarts = 2
criterion = MCRMSELoss(weighted=True)
eval_criterion = MCRMSELoss()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_scored = 68

In [5]:
token2int = {x:i for i, x in enumerate('().ACGUBEHIMSX')}

def preprocess_inputs(df, cols=['sequence', 'structure', 'predicted_loop_type']):
    return np.transpose(
        np.array(
            df[cols]
            .applymap(lambda seq: [token2int[x] for x in seq])
            .values
            .tolist()
        ),
        (0, 2, 1)
    )

In [6]:
def get_augment_data(df):
    # split augmented data into train and test
    df_augment = augment[augment['id'].isin(df['id'])]
    
    # sort augmented data by train id
    df_augment = df_augment.set_index('id')
    df_augment = df_augment.reindex(index=df['id'])
    df_augment = df_augment.reset_index()
    
    return df_augment

In [7]:
train = pd.read_json('/kaggle/input/stanford-covid-vaccine/train.json', lines=True)
test = pd.read_json('/kaggle/input/stanford-covid-vaccine/test.json', lines=True)
ss = pd.read_csv('/kaggle/input/stanford-covid-vaccine/sample_submission.csv')
augment = pd.read_csv('../input/how-to-generate-augmentation-data/augment_df.csv')

In [8]:
train_noise = train[train.signal_to_noise<1]
w_trn_noise = np.log(train_noise.signal_to_noise+1.1) / 2
train = train = train[train.signal_to_noise>=1]
w_trn = np.log(train.signal_to_noise+1.1) / 2

In [9]:
kmeans_model = KMeans(n_clusters=200, random_state=110).fit(preprocess_inputs(train)[:, :, 0])
groups = kmeans_model.labels_
gkf = GroupKFold(n_splits=nfolds)

In [10]:
train_augment = get_augment_data(train)
test_augment = get_augment_data(test)

In [11]:
for col in ['sequence', 'structure', 'predicted_loop_type']:
    test[col] = test[col].str[:107]

In [12]:
def compute_base_pairings(arr):
    pairings = np.zeros((len(arr), 107, 1), dtype=np.int8)
    
    for i in range(len(arr)):
        temp_struc = arr[i, :, 1]
        temp_nucle = arr[i, :, 0]
        temp = np.zeros(len(temp_struc), dtype=np.int8)
        temp_pair = np.zeros(len(temp_struc), dtype=np.int8)
        
        p = []
        count = 0
        
        # map pairings to same integer
        for j in range(len(temp_struc)):
            if temp_struc[j] == 0:
                count += 1
                p.append(count)
                temp[j] = count
            elif temp_struc[j] == 1:
                temp[j] = p.pop()
        
        # map pair nucleotide
        for j in range(1, count+1):
            pair_idx = np.where(temp==j)[0]
            
            if len(pair_idx) > 1:
                temp_pair[pair_idx[0]] = temp_nucle[pair_idx[1]]
                temp_pair[pair_idx[1]] = temp_nucle[pair_idx[0]]
            else:
                temp_pair[pair_idx[0]] = 3
    
        pairings[i, :, 0] = temp_pair
    
    return pairings

In [13]:
def get_paired_prob(df):
    bbp = np.zeros((len(df), 107, 3))
    bpps_nb_mean = 0.077522 # mean of bpps_nb across all training data
    bpps_nb_std = 0.08914   # std of bpps_nb across all training data

    for i, id in tqdm(enumerate(df.id.values)):
        probability = np.load(data_dir+'/bpps/%s.npy'%id)[:107, :107]
        bpps_nb = (probability > 0).sum(axis=0) / probability.shape[0]
        
        bbp[i, :, 0] = 1-probability.sum(axis=1)
        bbp[i, :, 1] = probability.max(axis=1)
        bbp[i, :, 2] = (bpps_nb - bpps_nb_mean) / bpps_nb_std

    return bbp

In [14]:
def prepare_df(df, mode='train'):
    inp_cat = preprocess_inputs(df)
    inp_pairs = compute_base_pairings(inp_cat)
    inp_cat = np.concatenate((inp_cat, inp_pairs), axis=2)
    
    inp_cont = get_paired_prob(df)
    
    if mode == 'train':
        tar = np.array(df[pred_cols].values.tolist()).transpose((0, 2, 1))
        
        return inp_cat, inp_cont, tar
    else:
        return inp_cat, inp_cont

In [15]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)

In [16]:
# prepare train and augmented train
train_cat, train_cont, train_tar = prepare_df(train)
train_cat_aug, train_cont_aug, train_tar_aug = prepare_df(train)

# prepare noisy data
train_cat_noise, train_cont_noise, train_tar_noise = prepare_df(train_noise)

# prepare test and augmented test
test_cat, test_cont = prepare_df(test, mode='test')
test_cat_aug, test_cont_aug = prepare_df(test, mode='test')

In [17]:
class CovidModel(nn.Module):
    def __init__(self, hidden_dim=512, embed_dim=100, seq_len=107, dropout=.5, num_gru_layers=2):
        super(CovidModel, self).__init__()
        self.inp_embedding = nn.Embedding(len(token2int), embed_dim)
        self.cont_embedding = nn.Linear(3, embed_dim*2)
        self.gru = nn.LSTM(embed_dim*6, hidden_dim, num_gru_layers, bidirectional=True, batch_first=True, dropout=dropout)
        self.dense1 = nn.Linear(hidden_dim*2, hidden_dim*2)
        self.dense2 = nn.Linear(hidden_dim*2, hidden_dim*2)
        self.out = nn.Linear(hidden_dim*2, len(pred_cols))
    
    def forward(self, cat, cont):
        x_cat = self.inp_embedding(cat)
        x_cont = self.cont_embedding(cont)
        x_cat = x_cat.view(x_cat.shape[0], x_cat.shape[1], x_cat.shape[2]*x_cat.shape[3])
        x = torch.cat((x_cat, x_cont), dim=2)
        x, _ = self.gru(x)
        x = self.dense1(x)
        x = F.relu(x)
        x = self.dense2(x)
        x = F.relu(x)
        x = self.out(x)
        
        return x

In [18]:
class CovidDataset(Dataset):
    def __init__(self, cat, cont, targets, weights=None, mode='train'):
        self.mode = mode
        self.data_cat = cat[:, :107, :]
        self.data_cont = cont
        
        if mode == 'train':
            self.weights = weights
            self.targets = targets
    
    def __len__(self):
        return len(self.data_cat)
    
    def __getitem__(self, idx):
        if self.mode == 'train':
            if self.weights is None:
                return torch.LongTensor(self.data_cat[idx]), torch.FloatTensor(self.data_cont[idx]), 0, torch.FloatTensor(self.targets[idx])
            return torch.LongTensor(self.data_cat[idx]), torch.FloatTensor(self.data_cont[idx]), torch.from_numpy(np.array(self.weights[idx])).float(), torch.FloatTensor(self.targets[idx])
        elif self.mode == 'test':
            return torch.LongTensor(self.data_cat[idx]), torch.FloatTensor(self.data_cont[idx]), 0, 0

In [19]:
for seed in range(nstarts):
    print(f'Train seed {seed}')
    set_seed(seed)
    
    for n, (tr, te) in enumerate(gkf.split(train_cat, train_tar, groups=groups)):
        print(f'Train fold {n+1}')
        # split data
        xtrain_cat, xval_cat = train_cat[tr], train_cat[te]
        xtrain_cont, xval_cont = train_cont[tr], train_cont[te]
        ytrain, yval = train_tar[tr], train_tar[te]
        
        # split augmented data
        xtrain_cat_aug, xval_cat_aug = train_cat_aug[tr], train_cat_aug[te]
        xtrain_cont_aug, xval_cont_aug = train_cont_aug[tr], train_cont_aug[te]
        ytrain_aug, yval_aug = train_tar_aug[tr], train_tar_aug[te]
        
        # add augmented training data to training data
        # xtrain_cat = np.concatenate((xtrain_cat, xtrain_cat_aug), axis=0)
        # xtrain_cont = np.concatenate((xtrain_cont, xtrain_cont_aug), axis=0)
        # ytrain = np.concatenate((ytrain, ytrain_aug), axis=0)
        # -> individual cv scores decrease a lot so not using it for now
        
        # add noisy data
        xtrain_cat = np.concatenate((xtrain_cat, train_cat_noise), axis=0)
        xtrain_cont = np.concatenate((xtrain_cont, train_cont_noise), axis=0)
        xtrain_weights = np.concatenate((w_trn, w_trn_noise), axis=0)
        ytrain = np.concatenate((ytrain, train_tar_noise), axis=0)
        
        train_set = CovidDataset(xtrain_cat, xtrain_cont, ytrain, weights=xtrain_weights)
        val_set = CovidDataset(xval_cat, xval_cont, yval, weights=None)
        
        dataloaders = {
            'train': DataLoader(train_set, batch_size=batch_size, shuffle=True),
            'val': DataLoader(val_set, batch_size=val_batch_size, shuffle=False)
        }
        
        model = CovidModel().to(device)
        checkpoint_path = f'repeat:{seed}_Fold:{n+1}.pt'
        optimizer = optim.Adam(model.parameters())
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, eps=1e-4, verbose=True)
        best_loss = {'train': np.inf, 'val': np.inf}
        
        for epoch in range(nepochs):
            t0 = time.time()
            epoch_loss = {'train': 0.0, 'val': 0.0}
            
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()
                else:
                    model.eval()
                
                running_loss = 0.0
                
                for i, (cat, cont, w, y) in enumerate(dataloaders[phase]):
                    cat, cont, w, y = cat.to(device), cont.to(device), w.to(device), y.to(device)
                    
                    optimizer.zero_grad()
                    
                    with torch.set_grad_enabled(phase=='train'):
                        preds = model(cat, cont)
                        preds = torch.reshape(preds[:, :train_scored], (len(preds)*train_scored,len(pred_cols)))
                        y = torch.reshape(y, (len(y)*train_scored, len(pred_cols)))
                        if phase == 'train':
                            loss = criterion(preds, y, weights=w)
                            loss.backward()
                            optimizer.step()
                        elif phase == 'val':
                            loss = eval_criterion(preds, y)
                    
                    running_loss += loss.item() / len(dataloaders[phase])
                
                epoch_loss[phase] = running_loss
            
            print("Epoch {}/{}   -   loss: {:5.5f}   -   val_loss: {:5.5f}   -   time: {:5.2f}s".format(
                epoch + 1, nepochs, epoch_loss['train'], epoch_loss['val'], time.time() - t0))
            
            scheduler.step(epoch_loss['val'])
            
            if epoch_loss['val'] < best_loss['val']:
                best_loss = epoch_loss
                torch.save(model.state_dict(), checkpoint_path)

Train seed 0
Train fold 1
Epoch 1/100   -   loss: 0.31759   -   val_loss: 0.46697   -   time:  3.45s
Epoch 2/100   -   loss: 0.27006   -   val_loss: 0.38263   -   time:  3.26s
Epoch 3/100   -   loss: 0.24413   -   val_loss: 0.34239   -   time:  3.29s
Epoch 4/100   -   loss: 0.22708   -   val_loss: 0.33005   -   time:  3.25s
Epoch 5/100   -   loss: 0.22022   -   val_loss: 0.31575   -   time:  3.24s
Epoch 6/100   -   loss: 0.21463   -   val_loss: 0.30889   -   time:  3.23s
Epoch 7/100   -   loss: 0.21070   -   val_loss: 0.30632   -   time:  3.29s
Epoch 8/100   -   loss: 0.20530   -   val_loss: 0.29320   -   time:  3.24s
Epoch 9/100   -   loss: 0.20153   -   val_loss: 0.28618   -   time:  3.25s
Epoch 10/100   -   loss: 0.19928   -   val_loss: 0.28234   -   time:  3.24s
Epoch 11/100   -   loss: 0.19640   -   val_loss: 0.28045   -   time:  3.24s
Epoch 12/100   -   loss: 0.19456   -   val_loss: 0.28224   -   time:  3.23s
Epoch 13/100   -   loss: 0.18782   -   val_loss: 0.27542   -   time:  3

In [20]:
oof = np.zeros((len(train)*68, nstarts, len(pred_cols)))
oof_targets = np.zeros((len(train)*68, len(pred_cols)))
preds = np.zeros((len(test)*107, len(pred_cols)))

In [21]:
def mean_mse_loss(y_true, y_pred, scored_indices=[0,1,3]):
    metrics = []
    
    for i in range(len(pred_cols)):
        if i in scored_indices:
            metrics.append(mean_squared_error(y_true[:, i], y_pred[:, i], squared=False))
    
    return np.mean(metrics)

In [22]:
for seed in range(nstarts):
    print(f"Inference for seed {seed}")
    seed_targets = []
    seed_oof = []
    seed_preds = np.zeros((len(test)*107, len(pred_cols), nfolds))
    
    for n, (tr, te) in enumerate(gkf.split(train_cat, train_tar, groups=groups)):
        # split data
        xval_cat, xval_cont, yval = train_cat[te], train_cont[te], train_tar[te]
        
        fold_preds = []
        
        val_set = CovidDataset(xval_cat, xval_cont, yval)
        test_set = CovidDataset(test_cat, test_cont, None, mode='test')
        
        dataloaders = {
            'val': DataLoader(val_set, batch_size=val_batch_size, shuffle=False),
            'test': DataLoader(test_set, batch_size=val_batch_size, shuffle=False)
        }
        
        checkpoint_path = f'repeat:{seed}_Fold:{n+1}.pt'
        model = CovidModel().to(device)
        model.load_state_dict(torch.load(checkpoint_path))
        model.eval()
        
        for phase in ['val', 'test']:
            for i, (cat, cont, w, y) in enumerate(dataloaders[phase]):
                if phase == 'val':
                    cat, cont, y = cat.to(device), cont.to(device), y.to(device)
                elif phase == 'test':
                    cat, cont = cat.to(device), cont.to(device)

                with torch.no_grad():
                    batch_preds = model(cat, cont)

                    if phase == 'val':
                        seed_targets.append(y)
                        seed_oof.append(batch_preds)
                    elif phase == 'test':
                        fold_preds.append(batch_preds)
        
        fold_preds = torch.cat(fold_preds, dim=0).cpu().numpy()
        fold_preds = np.reshape(fold_preds, (len(test)*107, len(pred_cols)))
        seed_preds[:, :, n] = fold_preds
    
    seed_targets = torch.cat(seed_targets, dim=0).cpu().numpy()
    seed_targets = np.reshape(seed_targets, (len(train)*68, len(pred_cols)))
    seed_oof = torch.cat(seed_oof, dim=0).cpu().numpy()
    seed_oof = seed_oof[:, :68, :]
    seed_oof = np.reshape(seed_oof, (len(train)*68, len(pred_cols)))
    seed_preds = np.mean(seed_preds, axis=2)
    
    print("Score for this seed {:5.5f}".format(mean_mse_loss(seed_targets, seed_oof)))
    oof_targets = seed_targets
    oof[:, seed, :] = seed_oof
    preds += seed_preds / nstarts

oof = np.mean(oof, axis=1)
print("\nOverall score is {:5.5f}".format(mean_mse_loss(oof_targets, oof)))

Inference for seed 0
Score for this seed 0.25272
Inference for seed 1
Score for this seed 0.25247

Overall score is 0.24706


In [23]:
ss['id'] = ss['id_seqpos'].apply(lambda x: "_".join(x.split('_')[:2])) # get seq id
scored_ids = ss.groupby('id').head(107).id_seqpos.values # get first n entries of id --> set them to preds and leave the rest at 0
ss.loc[ss.id_seqpos.isin(scored_ids), pred_cols] = preds
ss.drop('id', axis=1, inplace=True)

In [24]:
ss.to_csv('submission.csv', index=False)